## CMPE 251 Course Project
### Zaytung vs Cumhuriyet Fake News Detector

## Group Name: 
    Placeholder
## Group Members:
    Hasan Kemik 116207076
    Ali Çağan Keskin 116200079
 
## Data Collection, Cleaning, Tokenizing and Saving

### Importing Necessary Libraries

In [1]:
from lxml import html
import requests # requests is a simple HTTP library for Python, built for humans to read the data.
import pandas as pd # pandas helps us to carry out our entire data analysis workflow.
from time import sleep # provides us to take data without error.
# importing nltk and tokenizer.
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hasankemik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Zaytung

### Defining Functions for Zaytung

In [2]:
# removeList1: helps us to remove first 36 useless items of the Zaytung html, which are links like "Spor" "Gündem" etc.
def removeList1(page):
    i=0
    for c in range(36):
        page.remove(page[c-i])
        i = i + 1
    return page
# removeList2: helps us to remove first 10 useless items of the Zaytung html.
def removeList2(page):
    i=0
    for c in range(10):
        page.remove(page[c-i])
        i = i + 1
    return page

def removeOthers_1(page):
    i=0
    for c in range(36):
        page.remove(page[c-i])
        i = i + 1
    return page
# removeOthers_1_1: helps us to remove last 33 useless items of the Zaytung html, like page numbers and other links.
def removeOthers_1_1(page):
    for c in range(33):
        page.remove(page[-1])
    return page
# removeDevamı: helps us to remove the useless data 'devamı...' of the Zaytung html.
def removeDevamı(page):
    for c in page:
        if c == "devamı... ":
            page.remove(c)
    return page

def removeOthers_2(page):
    i = 0
    for c in range(10):
        page.remove(page[c-i])
        i = i + 1
    return page
    

In [3]:
# removeWordsZay: helps us to remove the useless strings '\n','\t','\r','\xa0','\' of the Zaytung data and replace with empty string.
def removeWordsZay(resultList):
    resultList = [w.replace('\n', "") for w in resultList]
    resultList = [w.replace('\t', "") for w in resultList]
    resultList = [w.replace('\r', "") for w in resultList]
    resultList = [w.replace('\xa0', "") for w in resultList]
    resultList = [w.replace("'\'", "") for w in resultList]
    return resultList

In [4]:
def controlPages(page):
    df2 = pd.DataFrame([['','','']],columns= ('Header', 'Data','Flag'))
    # for this method we will assign a DataFrame named 'df2' for each "news" comprising an'Header', 'Data','Flag'.
    # 'Header' is just header of the news,
    # 'Data' is a short summary of the news,
    # 'Flag' is the trueness of the news,
    # Since that this is a fake news detector program, 1 is for Fake News and 0 is for Accurate News.
    
    if(page == "sporindex.asp" or page == "digerleri.asp"): # Taking the 'Spor' and 'Digerleri' pages of Zaytung.
        my_page = "http://zaytung.com/" + page + "?pg=1" # we create a page value named 'my_page' to make some operations to link.
        pages = requests.get(my_page, timeout = 10) # we create a 'pages' object and get the data from 'my_page' and assign 10 s of timeout.
        tree = html.fromstring(pages.content) # taking html parts of pages object. 
        theXpath = '//a/text()' # we form an Xpath to get the direction of the data in the html.
        resultList = tree.xpath(theXpath) # we assign 'theXpath' values in tree to 'resultList', we're collecting our header data in this line
        theXpath2 = '//font//font/text()' # we form an 'theXpath2' to get news' summary data from the html.
        resultList2 = tree.xpath(theXpath2) # we assign 'theXpath2' values in tree to 'resultList2'.
        page_number = int(resultList[-1]) # we're checking how many pages does this link contains.
        print(page_number) # to see the number of page.
        #change the 25 to page_number if you want to collect all of the data from zaytung.
        for i in range(25): 
            # We reach the inner sections of 'Spor' and 'Digerleri' pages in this part to collect maximum data.
            my_page = "http://zaytung.com/" + page + "?pg=" + str(i+1)
            print(my_page) # to see the page link as a feedback to control the statements.
            pages = requests.get(my_page, timeout = 5) # we create a 'pages' object and get the data from 'my_page' and assign 5 ms of timeout.
            tree = html.fromstring(pages.content)
            theXpath = '//a/text()'
            resultList = tree.xpath(theXpath)
            theXpath2 = '//font/text()'
            resultList2 = tree.xpath(theXpath2)
            #  cleaning the data with implemented methods we've just defined.
            resultList = removeOthers_1(resultList)  
            resultList = removeOthers_1_1(resultList)
            resultList = removeDevamı(resultList)    
            resultList2 = removeWordsZay(resultList2)
            resultList2 = removeOthers_2(resultList2)
            i = 0 
            while i < len(resultList2):
                df3 = pd.DataFrame([[resultList[i], resultList2[i], 1]],columns= ('Header','Data','Flag'))
                i = i + 1 
                df2 = pd.concat([df2,df3]) # we concatenate 2 dataframes as 1 dataframe.
        return df2
    else: # Taking the 'Sinema', 'Blog', 'Kitap', 'Oyun' sections of Zaytung.
        my_page = "http://zaytung.com/" + page
        print(my_page)
        pages = requests.get(my_page, timeout = 10)
        tree = html.fromstring(pages.content)
        theXpath = '//a/text()'
        resultList = tree.xpath(theXpath)
        theXpath2 = '//font/text()'
        resultList2 = tree.xpath(theXpath2)
        resultList = removeList1(resultList)
        resultList = removeDevamı(resultList)
        resultList.remove(resultList[-1])
        resultList2 = removeWordsZay(resultList2)
        resultList2 = removeList2(resultList2)
        df2 = pd.DataFrame([['','','']],columns= ('Header', 'Data','Flag'))
        i = 0
        while i < len(resultList):
            s = ""
            try:
                s = resultList2[i]
            except:
                s = ""
            df3 = pd.DataFrame([[resultList[i], s, 1]],columns= ('Header','Data','Flag'))
            
            i = i + 1
            df2 = pd.concat([df2,df3]) # we concatenate 2 different dataframes as 1 again.
    return df2

### Getting Data From Zaytung

In [5]:
# requesting page as an html file, and parsing it's contents.

page_links = {"sporindex.asp","indexsnm.asp","indexblog.asp","indexkitap.asp","indexgame.asp","digerleri.asp"}

# We will assign a DataFrame named 'df' for each "news" comprising an'Header', 'Data','Flag'.
    # 'Header' is just header of the news,
    # 'Data' is a short summary of the news,
    # 'Flag' is the trueness of the news.
df = pd.DataFrame([['','','']],columns= ('Header', 'Data','Flag'))

for pages in page_links:
    df = pd.concat([df,controlPages(pages)])

216
http://zaytung.com/digerleri.asp?pg=1
http://zaytung.com/digerleri.asp?pg=2
http://zaytung.com/digerleri.asp?pg=3
http://zaytung.com/digerleri.asp?pg=4
http://zaytung.com/digerleri.asp?pg=5
http://zaytung.com/digerleri.asp?pg=6
http://zaytung.com/digerleri.asp?pg=7
http://zaytung.com/digerleri.asp?pg=8
http://zaytung.com/digerleri.asp?pg=9
http://zaytung.com/digerleri.asp?pg=10
http://zaytung.com/digerleri.asp?pg=11
http://zaytung.com/digerleri.asp?pg=12
http://zaytung.com/digerleri.asp?pg=13
http://zaytung.com/digerleri.asp?pg=14
http://zaytung.com/digerleri.asp?pg=15
http://zaytung.com/digerleri.asp?pg=16
http://zaytung.com/digerleri.asp?pg=17
http://zaytung.com/digerleri.asp?pg=18
http://zaytung.com/digerleri.asp?pg=19
http://zaytung.com/digerleri.asp?pg=20
http://zaytung.com/digerleri.asp?pg=21
http://zaytung.com/digerleri.asp?pg=22
http://zaytung.com/digerleri.asp?pg=23
http://zaytung.com/digerleri.asp?pg=24
http://zaytung.com/digerleri.asp?pg=25
http://zaytung.com/indexkitap.

In [6]:
df

,Header,Data,Flag
0,,,
0,,,
0,THY'nin Öğretmenler İçin Yaptığı %25 İndirim K...,"Bartu Ben, yeni bir dizi. Bartu Küçükçağlayan ...",1
0,"Özel Güvenliği Geçemeyen Muhtar, Yine Siteye A...",Neredeyse her hafta canlı yayında yaptığı ekon...,1
0,"BluTV’nin Yeni Dizisi ‘’Bartu Ben’’, Başroldek...",Bir süredir kamuoyunda tartışmalara neden olan...,1
0,Her Hafta Berat Albayrak'ın Mimiklerine Maruz ...,Geçtiğimiz Eylül ayı içerisinde Hazine ve Mali...,1
0,"McKinsey, Yabancı Yatırımcılar İçin İlk Türkiy...","MHP Genel Başkanı Devlet Bahçeli'nin ""Enflasyo...",1
0,AK Parti Hükumetine Para Karşılığı Danışmanlık...,Bir grup MHP'linin dün gece yarısı Meral Akşen...,1
0,Enflasyonun %24'e Çıktığını Duyan Ülkücüler Bu...,Geçtiğimiz cuma günü 7.7 şiddetinde deprem ve ...,1
0,"Bahçeli, Adamlarını Gece Yarısı Meral Akşener'...",Cumhurbaşkanı Erdoğan tarafından marketlerdeki...,1


In [7]:
# organizing data indexes and dropping empty row.
df = df.reset_index(drop= True) 
df = df.reindex() # this is for modifying the index numbers for avoiding the potential errors because of the removed items.
# dropping the empty rows of dataframe caused by initializing them.
df = df.drop([0])
df = df.drop([1])
df = df.reset_index(drop = True)

In [8]:
df.head()

,Header,Data,Flag
0,THY'nin Öğretmenler İçin Yaptığı %25 İndirim K...,"Bartu Ben, yeni bir dizi. Bartu Küçükçağlayan ...",1
1,"Özel Güvenliği Geçemeyen Muhtar, Yine Siteye A...",Neredeyse her hafta canlı yayında yaptığı ekon...,1
2,"BluTV’nin Yeni Dizisi ‘’Bartu Ben’’, Başroldek...",Bir süredir kamuoyunda tartışmalara neden olan...,1
3,Her Hafta Berat Albayrak'ın Mimiklerine Maruz ...,Geçtiğimiz Eylül ayı içerisinde Hazine ve Mali...,1
4,"McKinsey, Yabancı Yatırımcılar İçin İlk Türkiy...","MHP Genel Başkanı Devlet Bahçeli'nin ""Enflasyo...",1


# Cumhuriyet

In [9]:
page_link = 'http://www.cumhuriyet.com.tr/gundem' # main link of the trustable news website 'Cumhuriyet'.
page = requests.get(page_link, timeout = 5) # we create a 'pages' object and get the data from 'page_link' and assign 5 ms of timeout.
tree = html.fromstring(page.content) # taking html file as a parsed content.

#/html/body/div[2]/div[5]/div/div[2]/ul/li[11]/a

theXpath3 = '/html/body/div[2]/div[5]/div/div[2]/ul/li[11]/a/@href' # writing the Xpath of the data.
resultList3 = tree.xpath(theXpath3) # we assign the data at the Xpath to a resultList

In [10]:
resultList3

['/gundem/37642']

In [11]:
last_page = int(resultList3[0].split("/")[-1]) # declaration of last page
last_page # controlling the value.

37642

In [12]:
theXpath4 = '//span/text()'
resultList4 = []

### Getting Data from Cumhuriyet

In [13]:
# take pages   
# change the 25 to last_page if you want to collect all data from cumhuriyet, or a page range.
for i in range(25):
    page_link = 'http://www.cumhuriyet.com.tr/gundem/' + str(i+1)
    sleep(5)
    print(page_link)
    page = requests.get(page_link, timeout = 5)
    tree = html.fromstring(page.content)
    resultList4.append(tree.xpath(theXpath4))
    

http://www.cumhuriyet.com.tr/gundem/1
http://www.cumhuriyet.com.tr/gundem/2
http://www.cumhuriyet.com.tr/gundem/3
http://www.cumhuriyet.com.tr/gundem/4
http://www.cumhuriyet.com.tr/gundem/5
http://www.cumhuriyet.com.tr/gundem/6
http://www.cumhuriyet.com.tr/gundem/7
http://www.cumhuriyet.com.tr/gundem/8
http://www.cumhuriyet.com.tr/gundem/9
http://www.cumhuriyet.com.tr/gundem/10
http://www.cumhuriyet.com.tr/gundem/11
http://www.cumhuriyet.com.tr/gundem/12
http://www.cumhuriyet.com.tr/gundem/13
http://www.cumhuriyet.com.tr/gundem/14
http://www.cumhuriyet.com.tr/gundem/15
http://www.cumhuriyet.com.tr/gundem/16
http://www.cumhuriyet.com.tr/gundem/17
http://www.cumhuriyet.com.tr/gundem/18
http://www.cumhuriyet.com.tr/gundem/19
http://www.cumhuriyet.com.tr/gundem/20
http://www.cumhuriyet.com.tr/gundem/21
http://www.cumhuriyet.com.tr/gundem/22
http://www.cumhuriyet.com.tr/gundem/23
http://www.cumhuriyet.com.tr/gundem/24
http://www.cumhuriyet.com.tr/gundem/25


In [14]:
# removeWordsZay: helps us to remove the useless strings '\n','\t','\r','\' of the Cumhuriyet data and replace with empty string.
def removeWordsCum(resultList): 
    resultList = [w.replace('\n', "") for w in resultList]
    resultList = [w.replace('\t', "") for w in resultList]
    resultList = [w.replace('\r', "") for w in resultList]
    resultList = [w.replace('/', "") for w in resultList]
    return resultList

# we remove the the first 48 useless items of Cumhuriyet html.
def removeDictCum(page):
    i=0
    for c in range(47):
        page.remove(page[c-i])
        i = i + 1
    return page

resultList5 = []
for pages in resultList4:
    pages = removeWordsCum(pages) # implementing the cleaning methods for Cumhuriyet
    pages = removeDictCum(pages)
    resultList5.append(pages[:][:30]) # taking the first 30 data because after 30, the links are useless after 30th index.

In [15]:
# we create a DataFrame for Cumhuriyet to organize the data
# the DataFrame construction should be like Zaytung DataFrame.
# 'Header' is just header of the news,
# 'Data' is a short summary of the news,
# 'Flag' is the trueness of the news.
df2 = pd.DataFrame([['','','']],columns= ('Header', 'Data','Flag')) 
i = 0
while i < len(resultList5):
    k = 0
    for page in range(15):
        df4 = pd.DataFrame([[resultList5[i][k], resultList5[i][k+1], 0]],columns= ('Header','Data','Flag'))
        df2 = pd.concat([df2,df4])
        k = k + 2
    i = i + 1

In [16]:
# organizing data indexes and dropping empty row.
# this is for modifying the index numbers for avoiding the potential errors because of the removed items.
df2 = df2.reset_index(drop= True)
df2 = df2.reindex() 
df2 = df2.drop([0])

In [17]:
# organizing index values.
df = pd.concat([df,df2])
df = df.reset_index(drop= True)
df = df.reindex()

In [18]:
# lowering the sentences.
df['Header'] = [heads.lower() for heads in df['Header']]
df['Data'] = [data.lower() for data in df['Data']]

### Importing StopWords From NLTK and Arranging It

In [19]:
# importing stopwords turkish dictionary from nltk corpus
# and removing and adding our own selected words for cleaning process.
from nltk.corpus import stopwords
stop_words = set(stopwords.words('turkish'))
stop_words.add("bir")
stop_words.add("bu")
stop_words.add("onlar")
stop_words.add("bunlar")
stop_words.add("bir şey")
stop_words.add("damga")
stop_words.add("vurmak")
stop_words.add("kırmak")
stop_words.add("sana")
stop_words.add("sen")
stop_words.add("seninle")
stop_words.add("onunla")
stop_words.add("dolar")
stop_words.add("euro")
stop_words.add("pound")
stop_words.add("son")
stop_words.add("hafta")
stop_words.add("sabah")
stop_words.add("akşam")
stop_words.add("iyi")
stop_words.add("de")
stop_words.add("dan")
stop_words.add("gelen")
stop_words.add("gelenler")
stop_words.add("başkaban")
stop_words.add("cumhurbaşkanı")
stop_words.add("benim")
stop_words.add("senin")
stop_words.add("ben")
stop_words.add("sonra")
stop_words.add("önce")
stop_words.add("gün")
stop_words.add("hafta")
stop_words.add("saat")
stop_words.add("yıl")
stop_words.remove("nasıl")
stop_words.remove("nerde")
stop_words.remove("niçin")
stop_words.remove("ya")
stop_words.remove("acaba")
stop_words.remove("birşey")
stop_words.remove("defa")
stop_words.remove("sanki")
stop_words.remove("şey")

### Tokenizing the sentences

In [20]:
df.Header = [word_tokenize(sent) for sent in df.Header] # tokenizing the header sentences to words

In [21]:
df.Data = [word_tokenize(data) for data in df.Data] # tokenizing the data sentences to wordss

### Clearing the data

In [22]:
# special cleaning methods for header and data
for sent in df.Header:
    for word in sent:
        if word in stop_words:
            sent.remove(word)
        elif len(word) <= 3:
            sent.remove(word)
for sent in df.Data:
    for word in sent:
        if word in stop_words:
            sent.remove(word)
        elif len(word) <= 3:
            sent.remove(word)

In [23]:
df

,Header,Data,Flag
0,"[thy'nin, öğretmenler, i̇çin, yaptığı, 25, i̇n...","[bartu, yeni, dizi, bartu, küçükçağlayan, yazd...",1
1,"[özel, güvenliği, geçemeyen, muhtar, yine, sit...","[neredeyse, hafta, canlı, yayında, yaptığı, ek...",1
2,"[blutv, nin, yeni, dizisi, bartu, ’, ’, ,, baş...","[süredir, kamuoyunda, tartışmalara, olan, ulus...",1
3,"[hafta, berat, albayrak'ın, mimiklerine, maruz...","[geçtiğimiz, eylül, içerisinde, hazine, maliye...",1
4,"[mckinsey, yabancı, yatırımcılar, i̇çin, i̇lk,...","[genel, başkanı, devlet, bahçeli'nin, enflasyo...",1
5,"[parti, hükumetine, para, karşılığı, danışmanl...","[grup, mhp'linin, gece, yarısı, meral, akşener...",1
6,"[enflasyonun, 24'e, çıktığını, duyan, ülkücüle...","[geçtiğimiz, cuma, günü, şiddetinde, deprem, h...",1
7,"[bahçeli, adamlarını, gece, yarısı, meral, akş...","[erdoğan, tarafından, marketlerdeki, anormal, ...",1
8,"[borç, bulabilme, ümidiyle, ve, almanya, i̇liş...","[ekonomideki, sarsıntının, giderek, şiddetlenm...",1
9,"[aylarca, amerika, bize, ekonomik, savaş, açtı...","[hazine, maliye, bakanı, berat, albayrak, bm, ...",1


In [24]:
# recalculating indexes after deletion operation
df = df.reset_index(drop= True)
df = df.reindex()

In [25]:
# checking if there's any empty news and data row, and if there's any deleting them.
for i in range(df.shape[0]):
    if(df.Header[i] == [] and df.Data[i] == []):
        df = df.drop([i])
df = df.reset_index(drop= True)
df = df.reindex()

In [26]:
# selecting feature colums to be treated as datas, and then creating a x object. 
feature_cols = ['Header','Data']
x = df.loc[:,feature_cols]
y = df.Flag

In [27]:
df.head()

,Header,Data,Flag
0,"[thy'nin, öğretmenler, i̇çin, yaptığı, 25, i̇n...","[bartu, yeni, dizi, bartu, küçükçağlayan, yazd...",1
1,"[özel, güvenliği, geçemeyen, muhtar, yine, sit...","[neredeyse, hafta, canlı, yayında, yaptığı, ek...",1
2,"[blutv, nin, yeni, dizisi, bartu, ’, ’, ,, baş...","[süredir, kamuoyunda, tartışmalara, olan, ulus...",1
3,"[hafta, berat, albayrak'ın, mimiklerine, maruz...","[geçtiğimiz, eylül, içerisinde, hazine, maliye...",1
4,"[mckinsey, yabancı, yatırımcılar, i̇çin, i̇lk,...","[genel, başkanı, devlet, bahçeli'nin, enflasyo...",1


In [28]:
df.tail()

,Header,Data,Flag
1723,"[gerçek, otopside, ortaya, çıktı, yaşlı, kadın...","[düzce'de, 2, önce, evinde, bulunan, miyaser, ...",0
1724,"[gerdirme, ameliyatından, video, paylaştı]","[katie, price, yüz, gerdirme, ameliyatı, sıras...",0
1725,"[serdar, aziz'in, menajerinden, açıklama, fene...","[kulübü, galatasaray'la, problemler, yaşayan, ...",0
1726,"[öso'dan, münbiç, cephe, hattına, yeni, takviye]","[özgür, suriye, ordusu, öso, ,, fırat, kalkanı...",0
1727,"[münbiç, yolunda, konvoy, cerablus'tan, yola, ...","[terör, örgütü, pkkpyd, nin, münbiç, ten, çeki...",0


### Saving the data

In [29]:
# saving collected and cleaned data to a pickle file.
x.to_pickle('x.pickle')
y.to_pickle('y.pickle')